# Student Performance - Feature Engineering

## Set Up Environment

In [1]:
# Import libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read CSV file into a dataframe.
scores = pd.read_csv('data/StudentsPerformance.csv')
scores.head(3)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93


In [3]:
x = pd.crosstab(index=scores['gender'], columns=scores['lunch'])
x['total'] = x.sum(axis=1)
for i in range(len(x.columns)-1):
    x[x.columns[i]] = x[x.columns[i]] / x['total']
x.drop(columns=['total'], inplace=True)
x

lunch,free/reduced,standard
gender,,
female,0.364865,0.635135
male,0.344398,0.655602


## Define Classes

In [4]:
class Group_Metrics:
    """
    The Group_Metrics class is for generating group metrics for a dataframe.  These metrics can be added to the original
    dataframe or used to create summary tables for specific groups.
    
    Attributes:
        self.data: The dataframe that is passed in that group metrics can be calculated from.
        self.group_cols: A list of categorical columns in the dataframe passed in for which group metrics can be calculated.
    """
    
    def __init__(self, df, group_cols, numeric_cols):
        """The constructor method takes in a dataframe and saves it as an attribute."""
        
        self.data = df
        self.group_cols = group_cols
        self.numeric_cols = numeric_cols
    
    def insert_group_metric(self, groups, metric_col, metric_name, func):
        """This method takes the dataframe attribute, calculates a metric for each group of a categorical column, and
        adds the resulting values as a new column in the dataframe."""
        
        group_values = self.data.pivot_table(index=groups, values=metric_col, aggfunc=func)[metric_col]
        if type(groups) == str:
            scores['{} {} {}'.format(metric_col, groups, metric_name)] = self.data.set_index(groups).index.map(group_values)
        else:
            scores['{} {} {}'.format(metric_col, groups, metric_name)] = self.data.set_index(
                list(groups)).index.map(group_values)
    
    def gen_summary_table(self, groups, metrics, metric_names, normalize=False):
        """This method generates a summary table for the groups passed in, including the count for each group, the
        distribution of categorical values for each group, and the metrics passed in on each numeric column for each
        group."""
        
        categorical_cols = [c for c in self.group_cols if c != groups]
        
        df_list = []
        
        initial_df = pd.DataFrame(pd.Series(self.data[groups].value_counts()))
        initial_df.rename(columns={groups: 'count'}, inplace=True)
        df_list.append(initial_df)
        
        for col in categorical_cols:
            group_values = pd.crosstab(index=self.data[groups], columns=self.data[col])
            
            if normalize:
                group_values['total'] = group_values.sum(axis=1)
                for i in range(len(group_values.columns)-1):
                    group_values[group_values.columns[i]] = group_values[group_values.columns[i]] / group_values['total']
                group_values.drop(columns=['total'], inplace=True)
            
            df_list.append(group_values)
        
        for col in self.numeric_cols:
            for i in range(len(metrics)):
                group_values = self.data.pivot_table(index=groups, values=col, aggfunc=metrics[i])
                group_values.rename(columns={col: '{} {}'.format(metric_names[i], col)}, inplace=True)
                df_list.append(group_values)
        
        summary_table = pd.concat(df_list, axis=1)
        return summary_table
    
    def _slice_df(self, filters):
        """This method subsets the dataframe attribute by the filters that are passed in."""
        
        cols = list(filters)
        vals = list(filters.values())
        subset_df = self.data
        for i in range(len(filters)):
            subset_df = subset_df[subset_df[cols[i]] == vals[i]]
        return subset_df

In [5]:
# Instantiate a Group_Metrics object.
groups = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
metric_cols = ['math score', 'reading score', 'writing score']
group_scores = Group_Metrics(scores, groups, metric_cols)

In [6]:
group_scores.data

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [7]:
# Define metrics.
metric_names = ['mean', 'median', 'min', 'max', 'stdev']
functions = [np.mean, np.median, np.min, np.max, np.std]

In [8]:
# Create summary table by gender.
group_scores.gen_summary_table('gender', functions, metric_names, normalize=True)

,count,group A,group B,group C,group D,group E,associate's degree,bachelor's degree,high school,master's degree,...,mean reading score,median reading score,min reading score,max reading score,stdev reading score,mean writing score,median writing score,min writing score,max writing score,stdev writing score
gender,,,,,,,,,,,,,,,,,,,,,
female,518,0.069498,0.200772,0.347490,0.249035,0.133205,0.223938,0.121622,0.181467,0.069498,...,72.608108,73,17,100,14.378245,72.467181,74,10,100,14.844842
male,482,0.109959,0.178423,0.288382,0.275934,0.147303,0.219917,0.114108,0.211618,0.047718,...,65.473029,66,23,100,13.931832,63.311203,64,15,100,14.113832


In [9]:
# Create summary table by race/ethnicity.
group_scores.gen_summary_table('race/ethnicity', functions, metric_names, normalize=True)

,count,female,male,associate's degree,bachelor's degree,high school,master's degree,some college,some high school,free/reduced,...,mean reading score,median reading score,min reading score,max reading score,stdev reading score,mean writing score,median writing score,min writing score,max writing score,stdev writing score
group C,319,0.564263,0.435737,0.244514,0.125392,0.200627,0.059561,0.216301,0.153605,0.357367,...,69.103448,71,17,100,13.997033,67.827586,68,10,100,14.983378
group D,262,0.492366,0.507634,0.190840,0.106870,0.167939,0.087786,0.255725,0.190840,0.362595,...,70.030534,71,31,100,13.895306,70.145038,72,32,100,14.367707
group B,190,0.547368,0.452632,0.215789,0.105263,0.252632,0.031579,0.194737,0.200000,0.363158,...,67.352632,67,24,97,15.177499,65.600000,67,15,96,15.625173
group E,140,0.492857,0.507143,0.278571,0.128571,0.157143,0.057143,0.250000,0.128571,0.292857,...,73.028571,74,26,100,14.874024,71.407143,72,22,100,15.113906
group A,89,0.404494,0.595506,0.157303,0.134831,0.202247,0.033708,0.202247,0.269663,0.404494,...,64.674157,64,23,100,15.543762,62.674157,62,19,97,15.468278


In [10]:
# Create summary table by parental level of education.
group_scores.gen_summary_table('parental level of education', functions, metric_names, normalize=True)

,count,female,male,group A,group B,group C,group D,group E,free/reduced,standard,...,mean reading score,median reading score,min reading score,max reading score,stdev reading score,mean writing score,median writing score,min writing score,max writing score,stdev writing score
some college,226,0.522124,0.477876,0.079646,0.163717,0.305310,0.296460,0.154867,0.349558,0.650442,...,69.460177,70.5,23,100,14.057049,68.840708,70.0,19,99,15.012331
associate's degree,222,0.522523,0.477477,0.063063,0.184685,0.351351,0.225225,0.175676,0.346847,0.653153,...,70.927928,72.5,31,100,13.868948,69.896396,70.5,35,100,14.311122
high school,196,0.479592,0.520408,0.091837,0.244898,0.326531,0.224490,0.112245,0.357143,0.642857,...,64.704082,66.0,24,99,14.132130,62.448980,64.0,15,100,14.085907
some high school,179,0.508380,0.491620,0.134078,0.212291,0.273743,0.279330,0.100559,0.340782,0.659218,...,66.938547,67.0,17,100,15.479295,64.888268,66.0,10,100,15.736197
bachelor's degree,118,0.533898,0.466102,0.101695,0.169492,0.338983,0.237288,0.152542,0.372881,0.627119,...,73.000000,73.0,41,100,14.285250,73.381356,74.0,38,100,14.728262
master's degree,59,0.610169,0.389831,0.050847,0.101695,0.322034,0.389831,0.135593,0.406780,0.593220,...,75.372881,76.0,42,100,13.775163,75.677966,75.0,46,100,13.730711


In [11]:
# Create summary table by lunch.
group_scores.gen_summary_table('lunch', functions, metric_names, normalize=True)

,count,female,male,group A,group B,group C,group D,group E,associate's degree,bachelor's degree,...,mean reading score,median reading score,min reading score,max reading score,stdev reading score,mean writing score,median writing score,min writing score,max writing score,stdev writing score
standard,645,0.510078,0.489922,0.082171,0.187597,0.317829,0.258915,0.153488,0.224806,0.114729,...,71.654264,72,26,100,13.830602,70.823256,72,22,100,14.339487
free/reduced,355,0.532394,0.467606,0.101408,0.194366,0.321127,0.267606,0.115493,0.216901,0.123944,...,64.653521,65,17,100,14.895339,63.022535,64,10,100,15.433823


In [12]:
# Create summary table by test preparation course.
group_scores.gen_summary_table('test preparation course', functions, metric_names, normalize=True)

,count,female,male,group A,group B,group C,group D,group E,associate's degree,bachelor's degree,...,mean reading score,median reading score,min reading score,max reading score,stdev reading score,mean writing score,median writing score,min writing score,max writing score,stdev writing score
none,642,0.520249,0.479751,0.090343,0.190031,0.314642,0.280374,0.124611,0.218069,0.112150,...,66.534268,67,17,100,14.463885,64.504673,65,10,100,14.999661
completed,358,0.513966,0.486034,0.086592,0.189944,0.326816,0.229050,0.167598,0.229050,0.128492,...,73.893855,75,37,100,13.638384,74.418994,76,36,100,13.375335


In [13]:
# Add group metrics for individual groups.
for group in groups:
    for m_col in metric_cols:
        for i in range(len(metric_names)):
            group_scores.insert_group_metric(group, m_col, metric_names[i], functions[i])

In [14]:
group_scores.data

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,math score gender mean,math score gender median,...,reading score test preparation course mean,reading score test preparation course median,reading score test preparation course min,reading score test preparation course max,reading score test preparation course stdev,writing score test preparation course mean,writing score test preparation course median,writing score test preparation course min,writing score test preparation course max,writing score test preparation course stdev
0,female,group B,bachelor's degree,standard,none,72,72,74,63.633205,65,...,66.534268,67,17,100,14.463885,64.504673,65,10,100,14.999661
1,female,group C,some college,standard,completed,69,90,88,63.633205,65,...,73.893855,75,37,100,13.638384,74.418994,76,36,100,13.375335
2,female,group B,master's degree,standard,none,90,95,93,63.633205,65,...,66.534268,67,17,100,14.463885,64.504673,65,10,100,14.999661
3,male,group A,associate's degree,free/reduced,none,47,57,44,68.728216,69,...,66.534268,67,17,100,14.463885,64.504673,65,10,100,14.999661
4,male,group C,some college,standard,none,76,78,75,68.728216,69,...,66.534268,67,17,100,14.463885,64.504673,65,10,100,14.999661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,63.633205,65,...,73.893855,75,37,100,13.638384,74.418994,76,36,100,13.375335
996,male,group C,high school,free/reduced,none,62,55,55,68.728216,69,...,66.534268,67,17,100,14.463885,64.504673,65,10,100,14.999661
997,female,group C,high school,free/reduced,completed,59,71,65,63.633205,65,...,73.893855,75,37,100,13.638384,74.418994,76,36,100,13.375335
998,female,group D,some college,standard,completed,68,78,77,63.633205,65,...,73.893855,75,37,100,13.638384,74.418994,76,36,100,13.375335


In [15]:
# Generate list with combinations of 2 categorical columns.
from itertools import combinations
group_combos = list(combinations(groups, 2))
group_combos

[('gender', 'race/ethnicity'),
 ('gender', 'parental level of education'),
 ('gender', 'lunch'),
 ('gender', 'test preparation course'),
 ('race/ethnicity', 'parental level of education'),
 ('race/ethnicity', 'lunch'),
 ('race/ethnicity', 'test preparation course'),
 ('parental level of education', 'lunch'),
 ('parental level of education', 'test preparation course'),
 ('lunch', 'test preparation course')]

In [16]:
# Add group metrics for 2-column groups.
for group in group_combos:
    for m_col in metric_cols:
        for i in range(len(metric_names)):
            group_scores.insert_group_metric(group, m_col, metric_names[i], functions[i])

In [17]:
group_scores.data

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,math score gender mean,math score gender median,...,"reading score ('lunch', 'test preparation course') mean","reading score ('lunch', 'test preparation course') median","reading score ('lunch', 'test preparation course') min","reading score ('lunch', 'test preparation course') max","reading score ('lunch', 'test preparation course') stdev","writing score ('lunch', 'test preparation course') mean","writing score ('lunch', 'test preparation course') median","writing score ('lunch', 'test preparation course') min","writing score ('lunch', 'test preparation course') max","writing score ('lunch', 'test preparation course') stdev"
0,female,group B,bachelor's degree,standard,none,72,72,74,63.633205,65,...,69.177033,70,26,100,13.752776,67.595694,69,22,100,14.238182
1,female,group C,some college,standard,completed,69,90,88,63.633205,65,...,76.215859,77,41,100,12.800101,76.766520,77,41,100,12.535017
2,female,group B,master's degree,standard,none,90,95,93,63.633205,65,...,69.177033,70,26,100,13.752776,67.595694,69,22,100,14.238182
3,male,group A,associate's degree,free/reduced,none,47,57,44,68.728216,69,...,61.602679,61,17,92,14.496271,58.736607,59,10,93,14.707226
4,male,group C,some college,standard,none,76,78,75,68.728216,69,...,69.177033,70,26,100,13.752776,67.595694,69,22,100,14.238182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,63.633205,65,...,76.215859,77,41,100,12.800101,76.766520,77,41,100,12.535017
996,male,group C,high school,free/reduced,none,62,55,55,68.728216,69,...,61.602679,61,17,92,14.496271,58.736607,59,10,93,14.707226
997,female,group C,high school,free/reduced,completed,59,71,65,63.633205,65,...,69.870229,71,37,100,14.150508,70.351145,72,36,100,13.850256
998,female,group D,some college,standard,completed,68,78,77,63.633205,65,...,76.215859,77,41,100,12.800101,76.766520,77,41,100,12.535017


In [18]:
# Save new dataframe as a CSV file.
group_scores.data.to_csv('data/StudentsPerformance_Polished.csv', index=False)